### Load the Commit Statistics
For this clustering we examined 39 GitHub projects containing a total of 2,629 commits. The projects all included the Fluence Marine library. Code for Fluence is generally written in React and Rust.

In [1]:
project_root_path = '../../..'

import sys
bob_lib_path = project_root_path + "/sandbox/python/bob/"
sys.path.append(bob_lib_path)

import commit_stat_engineering
import json

dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

### Commit Stats


In [2]:
print(all_stats[0].keys())

dict_keys(['commit', 'totalFiles', 'binFiles', 'textFiles', 'textLines', 'javascriptFiles', 'javascriptLines', 'javascriptFilePct', 'javascriptLinePct', 'rustFiles', 'rustLines', 'rustFilePct', 'rustLinePct', 'markdownFiles', 'markdownLines', 'markdownFilePct', 'markdownLinePct', 'jsonFiles', 'jsonLines', 'jsonFilePct', 'jsonLinePct', 'imgFiles', 'imgFilePct', 'lockFiles', 'lockLines', 'lockFilePct', 'lockLinePct', 'yarnFiles', 'yarnLines', 'yarnFilePct', 'yarnLinePct', 'htmlFiles', 'htmlLines', 'htmlFilePct', 'htmlLinePct', 'clojureFiles', 'clojureLines', 'clojureFilePct', 'clojureLinePct', 'shellFiles', 'shellLines', 'shellFilePct', 'shellLinePct', 'gitignoreFiles', 'gitignoreLines', 'gitignoreFilePct', 'gitignoreLinePct', 'noextbinFiles', 'noextbinFilePct', 'noexttextFiles', 'noexttextLines', 'noexttextFilePct', 'noexttextLinePct'])


### Load Labels

In [1]:
import pandas as pd

data = pd.read_csv(project_root_path + '/sandbox/data/bob/commit_labeling.csv', sep='\t')
data

NameError: name 'project_root_path' is not defined

### Link Labels to Stats

In [4]:
import numpy as np
def make_x(commit_stats):
    return [
        np.log1p(commit_stats['binFiles']),
        np.log1p(commit_stats['textFiles']),
        np.log1p(commit_stats['textLines']),
        np.log1p(commit_stats['javascriptFiles']),
        np.log1p(commit_stats['javascriptLines']),
        commit_stats['javascriptLinePct'],
        np.log1p(commit_stats['rustFiles']),
        np.log1p(commit_stats['rustLines']),
        commit_stats['rustLinePct'],
        np.log1p(commit_stats['markdownFiles']),
        np.log1p(commit_stats['markdownLines']),
        commit_stats['markdownLinePct'],
        np.log1p(commit_stats['jsonFiles']),
        np.log1p(commit_stats['jsonLines']),
        commit_stats['jsonLinePct'],
        np.log1p(commit_stats['imgFiles']),
        np.log1p(commit_stats['lockFiles']),
        np.log1p(commit_stats['lockLines']),
        commit_stats['lockLinePct'],
        np.log1p(commit_stats['yarnFiles']),
        np.log1p(commit_stats['yarnLines']),
        commit_stats['yarnLinePct'],
        np.log1p(commit_stats['htmlFiles']),
        np.log1p(commit_stats['htmlLines']),
        commit_stats['htmlLinePct'],
        np.log1p(commit_stats['clojureFiles']),
        np.log1p(commit_stats['clojureLines']),
        commit_stats['clojureLinePct'],
        np.log1p(commit_stats['shellFiles']),
        np.log1p(commit_stats['shellLines']),
        commit_stats['shellLinePct'],
        np.log1p(commit_stats['gitignoreFiles']),
        np.log1p(commit_stats['gitignoreLines']),
        commit_stats['gitignoreLinePct']
    ]


In [10]:
stats = {}
for commit in all_stats:
    stats[commit['commit']] = commit

X = []
Y = []
a = 0
for row in data.to_numpy():
    commit_id = row[0]
    row_labels = row[slice(2, 12)]
    x = make_x(stats[commit_id])
    #print(x)
    #print(a)
    X.append(make_x(stats[commit_id]))
    Y.append(row_labels.tolist())
    a += 1
#print(a)

#print(Y[slice(2)])
#print(Y[0][0])
#print(X[slice(2)])

In [11]:
from sklearn.datasets import make_multilabel_classification

sample_X, sample_y = make_multilabel_classification(n_classes=3, random_state=0)
print(sample_y[slice(2)])
print(Y[slice(2)])

[[0 1 0]
 [0 1 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

print(len(X))
print(len(Y))

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X, Y)
score = clf.score(X, Y)
Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

140
140
0.7214285714285714


/home/bob/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.33, random_state=3)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
# Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

0.5957446808510638


/home/bob/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
#print(X_test)
#print(Y_pred)
pairs = []
for i in range(0,len(X_test)):
    #print(Y_pred[i].tolist())
    pairs.append(Y_pred[i].tolist() + X_test[i])
# print(pairs)

print(len(X_train))
import pandas as pd
pd.DataFrame(pairs).to_csv("/home/bob/test.tsv", sep="\t")

93
